In [1]:
import pandas as pd

In [2]:
import sasoptpy as so

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
df = pd.read_csv('../data/input.csv')

In [5]:
df.head()

,Pos,ID,Name,BV,SV,Team,3_xMins,3_Pts,4_xMins,4_Pts,...,6_Pts,7_xMins,7_Pts,8_xMins,8_Pts,9_xMins,9_Pts,10_xMins,10_Pts,Elite%
0,M,1,Fábio Vieira,5.4,5.4,Arsenal,10,0.66,7,0.47,...,0.71,10,0.72,11,0.65,11,0.62,11,0.67,0.00
1,F,2,G.Jesus,6.9,6.9,Arsenal,1,0.10,33,2.06,...,2.86,35,2.93,35,2.30,37,2.29,35,2.29,0.00
2,D,3,Gabriel,6.0,6.0,Arsenal,88,4.38,84,3.06,...,4.72,80,4.67,79,3.65,77,3.04,75,3.06,0.45
3,F,4,Havertz,8.1,8.1,Arsenal,90,5.41,83,4.20,...,5.68,80,5.83,79,4.43,78,4.06,78,4.24,0.01
4,G,5,Hein,4.0,4.0,Arsenal,0,0.00,0,0.00,...,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0.00


In [6]:
gk_data = df[df['Pos'] == 'G'].copy().reset_index()

In [7]:
gk_data.head()

,index,Pos,ID,Name,BV,SV,Team,3_xMins,3_Pts,4_xMins,...,6_Pts,7_xMins,7_Pts,8_xMins,8_Pts,9_xMins,9_Pts,10_xMins,10_Pts,Elite%
0,4,G,5,Hein,4.0,4.0,Arsenal,0,0.00,0,...,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0.00
1,13,G,14,Ramsdale,4.5,4.5,Arsenal,2,0.07,2,...,0.20,5,0.23,6,0.24,7,0.26,7,0.28,0.00
2,14,G,15,Raya,5.5,5.5,Arsenal,93,4.07,93,...,4.36,90,4.30,89,3.73,88,3.42,88,3.48,0.09
3,38,G,39,Gauci,4.0,4.0,Aston Villa,0,0.00,0,...,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0.00
4,45,G,46,Marschall,4.0,4.0,Aston Villa,0,0.00,0,...,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0.00


In [8]:
model = so.Model(name='gk_model')

NOTE: Initialized model gk_model.


In [9]:
gk_data.set_index('index', inplace=True)

In [10]:
gk_data.head()

,Pos,ID,Name,BV,SV,Team,3_xMins,3_Pts,4_xMins,4_Pts,...,6_Pts,7_xMins,7_Pts,8_xMins,8_Pts,9_xMins,9_Pts,10_xMins,10_Pts,Elite%
index,,,,,,,,,,,,,,,,,,,,,
4,G,5,Hein,4.0,4.0,Arsenal,0,0.00,0,0.00,...,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0.00
13,G,14,Ramsdale,4.5,4.5,Arsenal,2,0.07,2,0.09,...,0.20,5,0.23,6,0.24,7,0.26,7,0.28,0.00
14,G,15,Raya,5.5,5.5,Arsenal,93,4.07,93,3.55,...,4.36,90,4.30,89,3.73,88,3.42,88,3.48,0.09
38,G,39,Gauci,4.0,4.0,Aston Villa,0,0.00,0,0.00,...,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0.00
45,G,46,Marschall,4.0,4.0,Aston Villa,0,0.00,0,0.00,...,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0.00


In [11]:
players = gk_data.index.tolist()

In [12]:
lineup = model.add_variables(players, name='lineup', vartype=so.binary)

In [13]:
bench = model.add_variables(players, name='bench', vartype=so.binary)

In [14]:
total_xp = so.expr_sum(lineup[p] * gk_data.loc[p, '3_Pts'] for p in players) + 0.1 * so.expr_sum(bench[p] * gk_data.loc[p, '3_Pts'] for p in players)

In [15]:
model.set_objective(total_xp, name='total_xp_obj', sense=so.maximize);

In [16]:
model.add_constraints((lineup[p] + bench[p] <= 1 for p in players), name='lineup_or_bench'); 

In [17]:
model.add_constraint(so.expr_sum(lineup[p] for p in players) == 1, name='single_lineup');

In [18]:
model.add_constraint(so.expr_sum(bench[p] for p in players) == 1, name='single_bench');

In [19]:
budget = 8.0
model.add_constraint(so.expr_sum((lineup[p] + bench[p]) * gk_data.loc[p, 'BV'] for p in players) <= budget, name='budget_constraint');

In [20]:
model.export_mps(fetch=True)

'NAME    gk_model                                                                 \nROWS                                                                             \n MAX    total_xp_obj                                                             \n L      lineup_or_bench[4]                                                       \n L      lineup_or_bench[13]                                                      \n L      lineup_or_bench[14]                                                      \n L      lineup_or_bench[38]                                                      \n L      lineup_or_bench[45]                                                      \n L      lineup_or_bench[46]                                                      \n L      lineup_or_bench[50]                                                      \n L      lineup_or_bench[54]                                                      \n L      lineup_or_bench[72]                                                      \n L 

In [21]:
model.set_objective(-total_xp, name='total_xp_obj', sense='N');

In [22]:
model.export_mps(filename='gk.mps')

In [23]:
command = 'cbc gk.mps solve solu solution.txt'

In [24]:
!{command}

Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Aug 20 2024 

command line - cbc gk.mps solve solu solution.txt (default strategy 1)
At line 1 NAME    gk_model
At line 2 ROWS
At line 81 COLUMNS
At line 378 RHS
At line 418 RANGES
At line 419 BOUNDS
At line 568 ENDATA
Problem gk_model has 77 rows, 148 columns and 444 elements
Coin0008I gk_model read with 0 errors
Continuous objective value is -0.076 - 0.00 seconds
Cgl0005I 2 SOS with 134 members
Cgl0004I processed model has 70 rows, 134 columns (134 integer (134 of which binary)) and 402 elements
Cutoff increment increased from 1e-05 to 0.000999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -0.076
Cbc0038I Before mini branch and bound, 134 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improve solution (0.02 seconds)
Cbc0038I After 0.02 seconds - Feasibility pump exiting with objective of -0.076 - took 0.00 seconds
Cbc0012I Integer solution of -0.076 foun

In [25]:
import re

In [26]:
with open('solution.txt', 'r') as f:
    content = f.read()

idxs = re.findall(r'\[(\d+)\]', content)

starting_gk_index = int(idxs[0])
bench_gk_index = int(idxs[1])

starting_gk = gk_data[gk_data.index == starting_gk_index].loc[starting_gk_index, 'Name']
bench_gk = gk_data[gk_data.index == bench_gk_index].loc[bench_gk_index, 'Name']

print(f'The optimal starting GK is {starting_gk}')
print(f'The optimal bench GK is {bench_gk}')

The optimal starting GK is Lumley
The optimal bench GK is Rushworth


In [27]:
martinez_data = gk_data[gk_data.index == starting_gk_index]
martinez_data

,Pos,ID,Name,BV,SV,Team,3_xMins,3_Pts,4_xMins,4_Pts,...,6_Pts,7_xMins,7_Pts,8_xMins,8_Pts,9_xMins,9_Pts,10_xMins,10_Pts,Elite%
index,,,,,,,,,,,,,,,,,,,,,
465,G,466,Lumley,4.0,4.0,Southampton,2,0.07,2,0.06,...,0.08,3,0.11,4,0.15,4,0.13,5,0.17,0.02


In [28]:
flekken_data = gk_data[gk_data.index == bench_gk_index] 
flekken_data

,Pos,ID,Name,BV,SV,Team,3_xMins,3_Pts,4_xMins,4_Pts,...,6_Pts,7_xMins,7_Pts,8_xMins,8_Pts,9_xMins,9_Pts,10_xMins,10_Pts,Elite%
index,,,,,,,,,,,,,,,,,,,,,
614,G,615,Rushworth,4.0,4.0,Brighton,2,0.06,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0.0


In [29]:
with open('solution.txt', 'r') as f:
    for v in model.get_variables():
        v.set_value(0)
    for line in f:
        if 'objective value' in line:
            continue
        words = line.split()
        var = model.get_variable(words[1])
        var.set_value(float(words[2]))

print("LINEUP")
for p in players:
    if lineup[p].get_value() > 0.5:
        print(p, gk_data.loc[p])

print("BENCH")
for p in players:
    if bench[p].get_value() > 0.5:
        print(p, gk_data.loc[p])

LINEUP
465 Pos                   G
ID                  466
Name             Lumley
BV                  4.0
SV                  4.0
Team        Southampton
3_xMins               2
3_Pts              0.07
4_xMins               2
4_Pts              0.06
5_xMins               2
5_Pts              0.09
6_xMins               2
6_Pts              0.08
7_xMins               3
7_Pts              0.11
8_xMins               4
8_Pts              0.15
9_xMins               4
9_Pts              0.13
10_xMins              5
10_Pts             0.17
Elite%             0.02
Name: 465, dtype: object
BENCH
614 Pos                 G
ID                615
Name        Rushworth
BV                4.0
SV                4.0
Team         Brighton
3_xMins             2
3_Pts            0.06
4_xMins             0
4_Pts             0.0
5_xMins             0
5_Pts             0.0
6_xMins             0
6_Pts             0.0
7_xMins             0
7_Pts             0.0
8_xMins             0
8_Pts             0.0
9_xMin